<a href="https://colab.research.google.com/github/piltom/materia_machinelearning/blob/main/chatbot_tp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot
Un chatbot es un software inteligente que es capaz de comunicarse y realizar acciones similares a las de un humano. El objetivo de este proyecto es construir un modelo que prediga respuestas usando patrones y respuestas predefinidos. Se le proporciona un archivo llamado intents.json que contiene estos patrones. Los archivos de palabras y clases se proporcionan como ayuda adicional. Siéntase libre de hacer un bot más complejo extendiendo el archivo de intenciones. 

#### Possible chat with your bot
<code>
You: Hello, how are you? 
Bot: Hi there, how can I help?
You: what can you do?
Bot: I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies
You: thanks
Bot: My pleasure
You: see ya. got to go!
Bot: See you
</code>

# Solución
Al archivo intents original se le agregaron más entradas en los patrones, para tener un poco más de datos para trabajar. Se intentó originalmente hacer clustering con los datos originales pero resultaban muy escasos y poco relacionados entre sí dentro de un mismo grupo. En otras palabras, dos elementos que deberían pertenecer a un mismo grupo se encontraban completamente separados.

In [2]:
# Load json file with answer patterns
import requests
import json
import pandas as pd

intents = json.loads(requests.get("https://raw.githubusercontent.com/piltom/materia_machinelearning/main/intents.json").text)
intents_df=pd.DataFrame(intents['intents']).explode('patterns').dropna(subset=['patterns'])

Primero se deben normalizar los datos de entrada, para esto se hacen tres cosas: Reemplazar por sinónimos, eliminar las comas y hacer "stemming". Esto último consiste en encontrar la raiz de una palabra, lo cual es útil para que, por ejemplo, una palabra en singular y otra en plural no cuenten como dos características distintas. El reemplazo por sinónimos se hace de forma muy ineficiente, pero para este ejemplo no es un problema. Idealmente debería hacerse con un diccionario.

In [3]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
syn_lists=[["hi", "hello", "sup", "hey","ciao", "hola"],
           ["bye", "tschuss", "goodbye"],
           ["help", "helpful", "support"]]
def replace_w_syn(y):
  for syn_list in syn_lists:
    if y.lower() in syn_list:
      return syn_list[0]
  return y

intents_df['patterns_token']=intents_df['patterns'].apply(lambda x : filter(None,x.split(" ")))

intents_df['patterns_token']=intents_df['patterns_token'].apply(lambda x : [replace_w_syn(y.strip(",")) for y in x])

intents_df['patterns_stemmed']=intents_df['patterns_token'].apply(lambda x : [stemmer.stem(y) for y in x])

intents_df['patterns_stemmed_sentence']=intents_df['patterns_stemmed'].apply(lambda x : " ".join(x))

Para extraer las características de los textos se calculan los valores TF-IDF, que son pesos asignados a las palabras en relacion a su frecuencia dentro de la oración e inversamente relacionado con la frecuencia media en todas las oraciones.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

extractor = TfidfVectorizer(stop_words=["the","list","is", "can", "you","me","open","find","is", "all","for","of", "nearby", "i", "how", "good"])
extractor.fit(intents_df.patterns_stemmed_sentence.values)
features = extractor.transform(intents_df.patterns_stemmed_sentence.values)

Usando estas features para entrenar un KMeans. Las posibles categorías son 9, sacando las categorías que no tienen patrones y que son para ingresar datos de búsqueda en el chat. Estas categorías no se implementaron ya que escapan la parte de clustering y tienen más que ver con la implementación de el chat en si.
A continuación se puede ver el resultado del agrupamiento, junto con el texto de entrada y el texto procesado que se usó para extraer las features.

In [5]:
from sklearn.cluster import KMeans

clusterer = KMeans(9, random_state=15)
arr=clusterer.fit_predict( features )
resultados=pd.DataFrame({'cluster':arr, 'input':intents_df.patterns.values, 'procesada':intents_df.patterns_stemmed_sentence})
resultados

,cluster,input,procesada
0,2,Hi there,hi there
0,8,How are you,how are you
0,2,Is anyone there?,is anyon there?
0,2,Hey,hi
0,2,Hola,hi
0,2,Hello,hi
0,2,Good day,good day
0,2,"Hello, good day",hi good day
0,2,"hi, good day",hi good day
0,8,"Hi, How are you?",hi how are you?


Este clustering da como resultado una homogeneidad de 88.9%

In [6]:
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
homogeneity_score(intents_df.tag, clusterer.predict(features))

0.8898914776075445

Y un score de completitud de 90.2%

In [7]:
completeness_score(intents_df.tag, clusterer.predict(features))

0.9021467187485696

El V-measure da entonces 89.5%

In [8]:
v_measure_score(intents_df.tag, clusterer.predict(features))

0.895977193118296

# Relacionando las etiquetas con los clusters


Para identificar cada cluster con las etiquetas originales, se toma de cada cluster las frases que lo componen y se elige la etiqueta predominante.

In [17]:
grouped=resultados.groupby('cluster')
cluster_tags=[]
for cluster_num, group in grouped:
  print(cluster_num)
  print(intents_df.tag[intents_df.patterns.isin(group['input'].values)].value_counts().idxmax())
  cluster_tags.append(intents_df.tag[intents_df.patterns.isin(group['input'].values)].value_counts().idxmax())


0
adverse_drug
1
blood_pressure_search
2
greeting
3
thanks
4
pharmacy_search
5
options
6
hospital_search
7
goodbye
8
greeting


# Chat de prueba


In [27]:
import random
def process_input(x):
  return " ".join([stemmer.stem(y) for y in [replace_w_syn(y) for y in x.split(" ")]])
def get_reply(x):
 tag=cluster_tags[(clusterer.predict(extractor.transform([process_input(y) for y in x])))[0]]
 for intent in intents['intents']:
   if intent['tag']==tag:
     return intent['responses'][random.randint(0, len(intent['responses'])-1)]


Hey, how are you?

In [29]:
print(get_reply(["Hey, how are you?"]))

Good to see you again


What is it that you do?

In [30]:
print(get_reply(["What is it that you do?"]))

I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies


I would like to track my blood pressure

In [33]:
print(get_reply(["I would like to track my blood pressure"]))

Please provide Patient ID


...

*procesar la id y dar resultado no está implementado*

...

Thanks for your help

In [34]:
print(get_reply(["Thanks for your help"]))

Any time!


Bye, have a good day

In [35]:
print(get_reply(["Bye, have a good day"]))

See you!


# Fallas
Hay dos categorías que se combinaron en una, la categoría "blood_pressure_search" y la "blood_pressure". Estas dos categorías están muy superpuestas en el espacio de palabras, por lo que el algoritmo no puede diferenciarlas. En realidad, originalmente la categoría "blood_pressure_search" no debería ser una posiblidad, ya que a su contexto se accede desde una respuesta a la categoría "blood_pressure". En este ejemplo no se implementaron contextos, por lo que pasa lo que pasa.

Por la poca cantidad de patrones que se tienen, es difícil que el algoritmo relacione frases que significan lo mismo pero tienen pocas o ninguna palabra en común.

El chatbot está incompleto, ya que la parte de contexto no está implementada. Eso ayudaría a generar un árbol de posibles intenciones, que seguramente afinaría más el criterio de selección (si estoy buscando hospitales y todavía no tengo resultado, sería raro que pregunte por otra cosa).